In [1]:

import sqlite3
from tkinter import *
from tkinter import messagebox
from datetime import datetime


# Connect to SQLite database
conn = sqlite3.connect('acharya_hostel.db')
c = conn.cursor()

# Create Students table
c.execute('''CREATE TABLE IF NOT EXISTS Students(
            AUID TEXT PRIMARY KEY,
            Name TEXT,
            HostelID TEXT)''')

# Create Incharge table
c.execute('''CREATE TABLE IF NOT EXISTS Incharge(
            ID TEXT PRIMARY KEY,
            Name TEXT,
            Password TEXT,
            Category TEXT)''')

# Create Complaints table
c.execute('''CREATE TABLE IF NOT EXISTS Complaints(
            ComplaintID INTEGER PRIMARY KEY AUTOINCREMENT,
            StudentAUID TEXT,
            Category TEXT,
            SubCategory TEXT,
            Description TEXT,
            Status TEXT,
            DateSubmitted TEXT,
            DateResolved TEXT,
            Rating INTEGER,
            FOREIGN KEY(StudentAUID) REFERENCES Students(AUID))''')

conn.commit()
print("✅ Database and tables created successfully!")

#Add Sample In-Charge Data 
c.execute("INSERT OR IGNORE INTO Incharge(ID, Name, Password, Category) VALUES ('mess01','Mess Incharge','mess123','Mess')")
c.execute("INSERT OR IGNORE INTO Incharge(ID, Name, Password, Category) VALUES ('room01','Room Incharge','room123','Room')")
c.execute("INSERT OR IGNORE INTO Incharge(ID, Name, Password, Category) VALUES ('hyg01','Hygiene Incharge','hyg123','Hygiene')")
conn.commit()
print("✅ Sample in-charge data added!")

# ---------------- Step 4: Student App ----------------
def student_app():
    global conn, c
    login_window = Tk()
    login_window.title("Student Login")
    login_window.geometry("300x150")

    Label(login_window, text="Enter Your Acharya AUID:").pack(pady=10)
    entry_auid = Entry(login_window)
    entry_auid.pack()

    def login():
        global conn, c
        student_auid = entry_auid.get().upper()

        # Validate AUID format
        if not student_auid.startswith("AIT25") or len(student_auid) != 12:
            messagebox.showerror("Error", "AUID must start with 'AIT25' and be 12 characters long")
            return

        # Check if student exists
        c.execute("SELECT * FROM Students WHERE AUID=?", (student_auid,))
        student = c.fetchone()
        if not student:
            # New student, insert with blank Name and HostelID
            c.execute("INSERT INTO Students(AUID, Name, HostelID) VALUES (?,?,?)", (student_auid, '', ''))
            conn.commit()

        login_window.destroy()
        open_student_dashboard(student_auid)

    Button(login_window, text="Login", command=login).pack(pady=10)
    login_window.mainloop()

# Student Dashboard 
def open_student_dashboard(student_auid):
    global conn, c
    dash = Tk()
    dash.title(f"Student Dashboard - AUID: {student_auid}")
    dash.geometry("850x650")

    # Fetch current student info
    c.execute("SELECT Name, HostelID FROM Students WHERE AUID=?", (student_auid,))
    student = c.fetchone()
    student_name = student[0]
    student_hostel = student[1]

    Label(dash, text="Acharya Hostel Complaint App", font=("Arial", 16)).pack(pady=10)

    # ---------------- Student Info Frame ----------------
    info_frame = Frame(dash)
    info_frame.pack(pady=10)

    Label(info_frame, text="Name:").grid(row=0, column=0)
    entry_name = Entry(info_frame)
    entry_name.grid(row=0, column=1)
    entry_name.insert(0, student_name)

    Label(info_frame, text="Room/Hostel Number:").grid(row=1, column=0)
    entry_hostel = Entry(info_frame)
    entry_hostel.grid(row=1, column=1)
    entry_hostel.insert(0, student_hostel)

    def update_info():
        name = entry_name.get().strip()
        hostel = entry_hostel.get().strip()
        if not name or not hostel:
            messagebox.showerror("Error", "Please enter both Name and Room/Hostel number")
            return
        c.execute("UPDATE Students SET Name=?, HostelID=? WHERE AUID=?", (name, hostel, student_auid))
        conn.commit()
        messagebox.showinfo("Success", "Info updated!")

    Button(info_frame, text="Update Info", command=update_info).grid(row=2, column=0, columnspan=2, pady=5)

    # Complaint Submission Frame 
    frame = Frame(dash)
    frame.pack(pady=10)

    Label(frame, text="Category:").grid(row=0, column=0)
    category_var = StringVar()
    category_menu = OptionMenu(frame, category_var, "Mess Related", "Room Related", "Hygiene Related")
    category_menu.grid(row=0, column=1)

    Label(frame, text="Sub-Category (if Room):").grid(row=1, column=0)
    sub_var = StringVar()
    sub_menu = OptionMenu(frame, sub_var, "Roommate Issues", "Furniture/Keys Issues", "Electricity Issues")
    sub_menu.grid(row=1, column=1)

    Label(frame, text="Complaint Description:").grid(row=2, column=0)
    desc_entry = Entry(frame, width=50)
    desc_entry.grid(row=2, column=1)

    def submit_complaint():
        global conn, c
        cat = category_var.get()
        subcat = sub_var.get() if cat == "Room Related" else ""
        desc = desc_entry.get()
        if not cat or not desc:
            messagebox.showerror("Error", "Please fill all required fields")
            return
        now = datetime.now().strftime("%Y-%m-%d %H:%M")
        c.execute("INSERT INTO Complaints(StudentAUID, Category, SubCategory, Description, Status, DateSubmitted) VALUES (?,?,?,?,?,?)",
                  (student_auid, cat, subcat, desc, "Received", now))
        conn.commit()
        messagebox.showinfo("Success", "Complaint Submitted!")
        desc_entry.delete(0, END)
        load_complaints()

    Button(frame, text="Submit Complaint", command=submit_complaint).grid(row=3, column=0, columnspan=2, pady=10)

    #  Complaint Status List
    table_frame = Frame(dash)
    table_frame.pack(pady=10)

    complaint_listbox = Listbox(table_frame, width=100)
    complaint_listbox.pack()

    def load_complaints():
        global conn, c
        complaint_listbox.delete(0, END)
        c.execute("SELECT ComplaintID, Category, SubCategory, Description, Status FROM Complaints WHERE StudentAUID=?", (student_auid,))
        for comp in c.fetchall():
            complaint_listbox.insert(END, f"ID:{comp[0]} | Category:{comp[1]} | SubCategory:{comp[2]} | Desc:{comp[3]} | Status:{comp[4]}")

    load_complaints()
    dash.mainloop()

# In-Charge App 
def incharge_app():
    global conn, c
    login_window = Tk()
    login_window.title("In-Charge Login")
    login_window.geometry("300x180")

    Label(login_window, text="In-Charge ID:").pack(pady=5)
    entry_id = Entry(login_window)
    entry_id.pack()
    Label(login_window, text="Password:").pack(pady=5)
    entry_pass = Entry(login_window, show="*")
    entry_pass.pack()

    def login():
        global conn, c
        incharge_id = entry_id.get()
        password = entry_pass.get()
        c.execute("SELECT * FROM Incharge WHERE ID=? AND Password=?", (incharge_id, password))
        incharge = c.fetchone()
        if incharge:
            login_window.destroy()
            open_incharge_dashboard(incharge_id, incharge[3])  # ID and Category
        else:
            messagebox.showerror("Error", "Invalid ID or Password")

    Button(login_window, text="Login", command=login).pack(pady=10)
    login_window.mainloop()

def open_incharge_dashboard(incharge_id, category):
    global conn, c
    dash = Tk()
    dash.title(f"In-Charge Dashboard - {category}")
    dash.geometry("900x600")

    Label(dash, text=f"{category} In-Charge Dashboard", font=("Arial", 16)).pack(pady=10)

    table_frame = Frame(dash)
    table_frame.pack(pady=10)

    complaint_listbox = Listbox(table_frame, width=120)
    complaint_listbox.pack()

    status_var = StringVar()
    OptionMenu(dash, status_var, "Working", "Pending", "Resolved").pack(pady=5)

    def load_complaints():
        global conn, c
        complaint_listbox.delete(0, END)
        c.execute("SELECT ComplaintID, StudentAUID, SubCategory, Description, Status FROM Complaints WHERE Category LIKE ?", (f"%{category}%",))
        for comp in c.fetchall():
            complaint_listbox.insert(END, f"ID:{comp[0]} | Student:{comp[1]} | SubCategory:{comp[2]} | Desc:{comp[3]} | Status:{comp[4]}")

    def update_status():
        global conn, c
        selected = complaint_listbox.curselection()
        if not selected:
            messagebox.showerror("Error", "Select a complaint first")
            return
        index = selected[0]
        item_text = complaint_listbox.get(index)
        complaint_id = int(item_text.split("|")[0].split(":")[1])
        new_status = status_var.get()
        now = datetime.now().strftime("%Y-%m-%d %H:%M") if new_status=="Resolved" else None
        c.execute("UPDATE Complaints SET Status=?, DateResolved=? WHERE ComplaintID=?", (new_status, now, complaint_id))
        conn.commit()
        load_complaints()
        messagebox.showinfo("Success", "Status updated!")

    Button(dash, text="Update Status", command=update_status).pack(pady=5)
    load_complaints()
    dash.mainloop()
    
# student_app()
# incharge_app()

✅ Database and tables created successfully!
✅ Sample in-charge data added!
